<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/Get_Zip_Code_Rental_Data_by_County.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Zip Code Rental Data by County

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 22nd Jun 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
from scipy.stats import zscore
import requests
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

warnings.filterwarnings('ignore')

## <font color="blue">Functions</font>

In [ ]:
def get_rental_market_data(rapid_api_key, zip_code):

  if len(str(zip_code)) != 5:
    return "Invalid Zip Code"

  url = "https://realty-mole-property-api.p.rapidapi.com/zipCodes/{}".format(str(zip_code))

  headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
  }

  return requests.request("GET", url, headers=headers)

In [ ]:
def move_col_to_front(df, col_name):
  front_col = df[col_name]
  df.drop(labels=[col_name], axis=1,inplace = True)
  df.insert(0, col_name, front_col)
  return df

In [ ]:
def get_latest_rental_data(response):
  # transform data to pandas dataframe
  df = pd.json_normalize(data=response.json())
  # relevant cols
  rent_cols = ['id', 'rentalData.averageRent', 'rentalData.minRent', 'rentalData.maxRent', 'rentalData.totalRentals', 'rentalData.detailed']
  df = df[rent_cols]
  return df.rename(columns={'id': 'zip_code'})

def get_historical_rental_data(response):
  # transform data to pandas dataframe
  df_rent = pd.json_normalize(data=response.json())

  """ get historical data to single dataframe """
  df_list = [] # create empty list
  # get all "detailed" columns
  rent_detail_hist_cols = [x for x in df_rent.columns if 'detailed' and 'history' in x]
  # iterate through "detailed" columns
  for x in rent_detail_hist_cols:
    # get column date
    date_str = x.split('.')[2]
    # get column name
    detail_col = 'rentalData.history.' + date_str + '.detailed'
    # convert historical data to a dataframe
    _df = pd.DataFrame(df_rent[detail_col].iloc[0])
    # create columns
    _df['date_str'] = date_str
    _df['zip_code'] = zip_code
    # append to list
    df_list.append(_df)

  """ combine """
  # comine sub date dataframes
  df_detail = pd.concat(df_list)
  # move date column to front
  df_detail = move_col_to_front(df=df_detail, col_name='date_str')
  df_detail = move_col_to_front(df=df_detail, col_name='zip_code')
  # add feature for previous year
  df_detail['prev_yr_dt'] = df_detail.apply(lambda x: (datetime.strptime(x['date_str'], '%Y-%m') - relativedelta(years=1)).strftime('%Y-%m'), axis=1)

  """ merge """
  df_detail_prev = df_detail.copy()
  df_detail_prev.columns = [x + '_prev_yr' for x in df_detail_prev.columns]
  # merge
  df_merge = pd.merge(df_detail, df_detail_prev, how='left', left_on=['prev_yr_dt', 'bedrooms'], right_on=['date_str_prev_yr', 'bedrooms_prev_yr']).drop_duplicates()
  # identify outliers
  df_merge['averageRent_zscore'] = df_merge[['averageRent']].apply(zscore)

  """ filter """
  # filter
  df_filter = df_merge.copy()
  df_filter = df_filter.loc[(df_filter['bedrooms'] <= 4) & (df_filter['averageRent_zscore'] <= 2)]
  df_filter = df_filter.drop(columns=['prev_yr_dt', 'date_str_prev_yr', 'prev_yr_dt_prev_yr', 'zip_code_prev_yr'])
  # add features
  df_filter['YoY_avg_rent'] = \
    (df_filter['averageRent'] - df_filter['averageRent_prev_yr']) / df_filter['averageRent_prev_yr']
  df_filter['YoY_total_rentals'] = \
    (df_filter['totalRentals'] - df_filter['totalRentals_prev_yr']) / df_filter['totalRentals_prev_yr']
  return df_filter

## <font color="blue">Locals & Constants</font>

In [ ]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

## <font color="blue">Transformations</font>

### <font color="green">1. Single Zip Code</font>

In [ ]:
zip_code = '33610' # Ybor, Tampa, FL
url = "https://realty-mole-property-api.p.rapidapi.com/zipCodes/{}".format(str(zip_code))

headers = {
  "X-RapidAPI-Key": rapid_api_key,
  "X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

In [ ]:
response.json()

{'id': '33610',
 'rentalData': {'averageRent': 21264.16,
  'detailed': [{'averageRent': 118059.5,
    'bedrooms': 0,
    'maxRent': 469000,
    'minRent': 769,
    'totalRentals': 4},
   {'averageRent': 1590.2,
    'bedrooms': 1,
    'maxRent': 1897,
    'minRent': 850,
    'totalRentals': 64},
   {'averageRent': 1825.64,
    'bedrooms': 2,
    'maxRent': 3000,
    'minRent': 985,
    'totalRentals': 70},
   {'averageRent': 2002.55,
    'bedrooms': 3,
    'maxRent': 3500,
    'minRent': 1350,
    'totalRentals': 96},
   {'averageRent': 2060.08,
    'bedrooms': 4,
    'maxRent': 2975,
    'minRent': 1200,
    'totalRentals': 26},
   {'averageRent': 2047,
    'bedrooms': 5,
    'maxRent': 2099,
    'minRent': 1995,
    'totalRentals': 2}],
  'history': {'2020-04': {'averageRent': 1367.13,
    'detailed': [{'averageRent': 826,
      'bedrooms': 0,
      'maxRent': 959,
      'minRent': 744,
      'totalRentals': 3},
     {'averageRent': 920,
      'bedrooms': 1,
      'maxRent': 1040,
   

In [ ]:
# transform data to pandas dataframe
df_rent = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_rent))
print('Num of cols:', len(df_rent.columns))
df_rent.head()

Num of rows: 1
Num of cols: 141


,id,rentalData.averageRent,rentalData.minRent,rentalData.maxRent,rentalData.totalRentals,rentalData.detailed,rentalData.history.2020-04.averageRent,rentalData.history.2020-04.minRent,rentalData.history.2020-04.maxRent,rentalData.history.2020-04.totalRentals,...,rentalData.history.2022-05.averageRent,rentalData.history.2022-05.minRent,rentalData.history.2022-05.maxRent,rentalData.history.2022-05.totalRentals,rentalData.history.2022-05.detailed,rentalData.history.2022-06.averageRent,rentalData.history.2022-06.minRent,rentalData.history.2022-06.maxRent,rentalData.history.2022-06.totalRentals,rentalData.history.2022-06.detailed
0,33610,21264.16,769,469000,262,"[{'bedrooms': 0, 'averageRent': 118059.5, 'min...",1367.13,744,49900,186,...,25126.24,769,469000,256,"[{'bedrooms': 0, 'averageRent': 141359.17, 'mi...",21264.16,769,469000,262,"[{'bedrooms': 0, 'averageRent': 118059.5, 'min..."


#### Get historical rental data

In [ ]:
df_list = [] # create empty list
# get all "detailed" columns
rent_detail_hist_cols = [x for x in df_rent.columns if 'detailed' and 'history' in x]
# iterate through "detailed" columns
for x in rent_detail_hist_cols:
  # get column date
  date_str = x.split('.')[2]
  # get column name
  detail_col = 'rentalData.history.' + date_str + '.detailed'
  # convert historical data to a dataframe
  _df = pd.DataFrame(df_rent[detail_col].iloc[0])
  # create columns
  _df['date_str'] = date_str
  _df['zip_code'] = zip_code
  # append to list
  df_list.append(_df)

In [ ]:
# comine sub date dataframes
df_detail = pd.concat(df_list)
# move date column to front
df_detail = move_col_to_front(df=df_detail, col_name='date_str')
df_detail = move_col_to_front(df=df_detail, col_name='zip_code')
# add feature for previous year
df_detail['prev_yr_dt'] = df_detail.apply(lambda x: (datetime.strptime(x['date_str'], '%Y-%m') - relativedelta(years=1)).strftime('%Y-%m'), axis=1)
df_detail.head()

,zip_code,date_str,bedrooms,averageRent,minRent,maxRent,totalRentals,prev_yr_dt
0,33610,2020-04,0,826.00,744,959,3,2019-04
1,33610,2020-04,1,920.00,800,1040,2,2019-04
2,33610,2020-04,2,1121.51,842,1450,35,2019-04
3,33610,2020-04,3,2103.31,950,49900,124,2019-04
4,33610,2020-04,4,1445.26,1100,2150,19,2019-04


#### Filter and add features

In [ ]:
df_detail_prev = df_detail.copy()
df_detail_prev.columns = [x + '_prev_yr' for x in df_detail_prev.columns]
# merge
df_merge = pd.merge(df_detail, df_detail_prev, how='left', left_on=['prev_yr_dt', 'bedrooms'], right_on=['date_str_prev_yr', 'bedrooms_prev_yr']).drop_duplicates()
# identify outliers
df_merge['averageRent_zscore'] = df_merge[['averageRent']].apply(zscore)
# view latest two months
df_merge.tail(10)

,zip_code,date_str,bedrooms,averageRent,minRent,maxRent,totalRentals,prev_yr_dt,zip_code_prev_yr,date_str_prev_yr,bedrooms_prev_yr,averageRent_prev_yr,minRent_prev_yr,maxRent_prev_yr,totalRentals_prev_yr,prev_yr_dt_prev_yr,averageRent_zscore
2135,33610,2022-05,2,1796.96,975,3000,69,2021-05,33610,2021-05,2.0,1219.70,800.0,1700.0,20.0,2020-05,-0.276613
2140,33610,2022-05,3,1943.76,1350,2950,94,2021-05,33610,2021-05,3.0,1479.40,1025.0,2700.0,45.0,2020-05,-0.272458
2145,33610,2022-05,4,2049.23,1200,2975,22,2021-05,33610,2021-05,4.0,1540.44,1395.0,1699.0,9.0,2020-05,-0.269473
2150,33610,2022-05,5,1995.00,1995,1995,1,2021-05,33610,2021-05,5.0,2322.50,1695.0,2950.0,2.0,2020-05,-0.271008
2275,33610,2022-06,0,118059.50,769,469000,4,2021-06,33610,2021-06,0.0,1018.00,822.0,1214.0,2.0,2020-06,3.014071
2280,33610,2022-06,1,1590.20,850,1897,64,2021-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.282465
2281,33610,2022-06,2,1825.64,985,3000,70,2021-06,33610,2021-06,2.0,1202.78,800.0,1700.0,18.0,2020-06,-0.275801
2286,33610,2022-06,3,2002.55,1350,3500,96,2021-06,33610,2021-06,3.0,1494.50,1025.0,2700.0,44.0,2020-06,-0.270794
2291,33610,2022-06,4,2060.08,1200,2975,26,2021-06,33610,2021-06,4.0,1563.00,1395.0,1699.0,8.0,2020-06,-0.269166
2296,33610,2022-06,5,2047.00,1995,2099,2,2021-06,33610,2021-06,5.0,2322.50,1695.0,2950.0,2.0,2020-06,-0.269536


In [ ]:
# filter
df_filter = df_merge.copy()
df_filter = df_filter.loc[(df_filter['bedrooms'] <= 4) & (df_filter['averageRent_zscore'] <= 2)]
df_filter = df_filter.drop(columns=['prev_yr_dt', 'date_str_prev_yr', 'prev_yr_dt_prev_yr', 'zip_code_prev_yr'])
# add features
df_filter['YoY_avg_rent'] = \
  (df_filter['averageRent'] - df_filter['averageRent_prev_yr']) / df_filter['averageRent_prev_yr']
df_filter['YoY_total_rentals'] = \
  (df_filter['totalRentals'] - df_filter['totalRentals_prev_yr']) / df_filter['totalRentals_prev_yr']
# view latest two months
df_filter.tail(10)

,zip_code,date_str,bedrooms,averageRent,minRent,maxRent,totalRentals,bedrooms_prev_yr,averageRent_prev_yr,minRent_prev_yr,maxRent_prev_yr,totalRentals_prev_yr,averageRent_zscore,YoY_avg_rent,YoY_total_rentals
1990,33610,2022-04,3,1961.48,1100,2950,91,3.0,1474.38,1025.0,2700.0,45.0,-0.271957,0.330376,1.022222
1995,33610,2022-04,4,2094.61,1200,2975,18,4.0,1534.89,1395.0,1699.0,9.0,-0.268188,0.364665,1.000000
2130,33610,2022-05,1,1613.31,850,2035,64,1.0,600.00,600.0,600.0,1.0,-0.281811,1.688850,63.000000
2135,33610,2022-05,2,1796.96,975,3000,69,2.0,1219.70,800.0,1700.0,20.0,-0.276613,0.473280,2.450000
2140,33610,2022-05,3,1943.76,1350,2950,94,3.0,1479.40,1025.0,2700.0,45.0,-0.272458,0.313884,1.088889
2145,33610,2022-05,4,2049.23,1200,2975,22,4.0,1540.44,1395.0,1699.0,9.0,-0.269473,0.330289,1.444444
2280,33610,2022-06,1,1590.20,850,1897,64,NaN,NaN,NaN,NaN,NaN,-0.282465,NaN,NaN
2281,33610,2022-06,2,1825.64,985,3000,70,2.0,1202.78,800.0,1700.0,18.0,-0.275801,0.517850,2.888889
2286,33610,2022-06,3,2002.55,1350,3500,96,3.0,1494.50,1025.0,2700.0,44.0,-0.270794,0.339946,1.181818
2291,33610,2022-06,4,2060.08,1200,2975,26,4.0,1563.00,1395.0,1699.0,8.0,-0.269166,0.318029,2.250000


### <font color="green">2. Multiple Zip Codes - All for a county</font>

#### Get county, zip data

In [ ]:
geo_data_url = 'https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv'
df_geo = pd.read_csv(geo_data_url)
print('Num of rows:', len(df_geo))
print('Num of columns:', len(df_geo.columns))
df_geo.head()

Num of rows: 33103
Num of columns: 6


,state_fips,state,state_abbr,zipcode,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


In [ ]:
# filter on single county
df_geo_county = df_geo.loc[(df_geo['state_abbr'] == 'FL') &
                           (df_geo['county'] == 'Hillsborough')]
# create feature to check if zip code is valid
df_geo_county['valid_zip_code'] = df_geo_county.apply(lambda x: x['zipcode'].isnumeric(), axis=1)
print('Dataset:')
print(df_geo_county.groupby(['valid_zip_code'])['zipcode'].count())
print(' ')
print('Invalid zip codes:', df_geo_county.loc[df_geo_county['valid_zip_code'] == False]['zipcode'].unique())

Dataset:
valid_zip_code
False     2
True     45
Name: zipcode, dtype: int64
 
Invalid zip codes: ['335HH' '336HH']


In [ ]:
# filter on only valid zip codes
df_geo_county_valid = df_geo_county.loc[df_geo_county['valid_zip_code'] == True]
print('Num of VALID zip codes:', len(df_geo_county_valid))
df_geo_county_valid.head()

Num of VALID zip codes: 45


,state_fips,state,state_abbr,zipcode,county,city,valid_zip_code
5129,12,Florida,FL,33510,Hillsborough,Brandon,True
5130,12,Florida,FL,33511,Hillsborough,Brandon,True
5136,12,Florida,FL,33527,Hillsborough,Dover,True
5137,12,Florida,FL,33534,Hillsborough,Gibsonton,True
5143,12,Florida,FL,33547,Hillsborough,Lithia,True


#### Get data for each zip code

In [ ]:
_df_latest_list = []
_df_historical_list = []

# iterate through each zip code in the list
for zip_code in df_geo_county_valid['zipcode'].tolist():
  print('Getting data for zipcode:', zip_code)
  
  # for each zip code get rental data
  response = get_rental_market_data(rapid_api_key, zip_code)

  try:

    # transform response
    # 1) latest rental data
    _df_latest = get_latest_rental_data(response)
    _df_latest_list.append(_df_latest)

    # 2) historical rental data
    _df_historical = get_historical_rental_data(response)
    _df_historical_list.append(_df_historical)

  except:

    print('**Error in retrieving data for:', zip_code)

  time.sleep(1) # pause for 2 seconds (max 2 requests per second)

Getting data for zipcode: 33510
Getting data for zipcode: 33511
Getting data for zipcode: 33527
Getting data for zipcode: 33534
Getting data for zipcode: 33547
Getting data for zipcode: 33549
Getting data for zipcode: 33556
Getting data for zipcode: 33565
Getting data for zipcode: 33566
Getting data for zipcode: 33567
Getting data for zipcode: 33569
Getting data for zipcode: 33570
Getting data for zipcode: 33572
Getting data for zipcode: 33573
Getting data for zipcode: 33584
Getting data for zipcode: 33592
Getting data for zipcode: 33594
Getting data for zipcode: 33598
Getting data for zipcode: 33602
Getting data for zipcode: 33603
Getting data for zipcode: 33604
Getting data for zipcode: 33605
Getting data for zipcode: 33606
Getting data for zipcode: 33607
Getting data for zipcode: 33609
Getting data for zipcode: 33610
Getting data for zipcode: 33611
Getting data for zipcode: 33612
Getting data for zipcode: 33613
Getting data for zipcode: 33614
Getting data for zipcode: 33615
Getting 

In [ ]:
# concat
df_latest = pd.concat(_df_latest_list)
df_latest.head()

,zip_code,rentalData.averageRent,rentalData.minRent,rentalData.maxRent,rentalData.totalRentals,rentalData.detailed
0,33510,2253.22,650,4300,243,"[{'bedrooms': 0, 'averageRent': 1597.33, 'minR..."
0,33511,2148.86,1100,4090,946,"[{'bedrooms': 0, 'averageRent': 1573.63, 'minR..."
0,33527,3901.67,2495,4950,10,"[{'bedrooms': 3, 'averageRent': 4950, 'minRent..."
0,33534,1785.64,800,2965,120,"[{'bedrooms': 0, 'averageRent': 1443.33, 'minR..."
0,33547,2222.97,1200,3645,176,"[{'bedrooms': 0, 'averageRent': 1624.46, 'minR..."


In [ ]:
# concat
df_historical = pd.concat(_df_historical_list)
df_historical.groupby(['zip_code'])['bedrooms'].count().reset_index().rename(columns={'bedrooms': 'num_records'})

,zip_code,num_records
0,33510,135
1,33511,135
2,33527,72
3,33534,125
4,33547,120
5,33549,129
6,33556,126
7,33565,68
8,33566,107
9,33567,78


## <font color="blue">Output</font>

In [ ]:
latest_dt_str = df_historical['date_str'].max()
print('Latest date:', latest_dt_str)

Latest date: 2022-06


In [ ]:
# download file
df_latest.to_csv('zip_rent_analysis_latest_{}.csv'.format(latest_dt_str), index=False)
files.download('zip_rent_analysis_latest_{}.csv'.format(latest_dt_str))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_historical_context = pd.merge(df_historical, df_geo_county_valid.rename(columns={'zipcode': 'zip_code'}), how='left', on='zip_code')
df_historical_context.head(1)

,zip_code,date_str,bedrooms,averageRent,minRent,maxRent,totalRentals,bedrooms_prev_yr,averageRent_prev_yr,minRent_prev_yr,...,totalRentals_prev_yr,averageRent_zscore,YoY_avg_rent,YoY_total_rentals,state_fips,state,state_abbr,county,city,valid_zip_code
0,33510,2020-04,0,979.25,730,1281,4,NaN,NaN,NaN,...,NaN,-1.217757,NaN,NaN,12,Florida,FL,Hillsborough,Brandon,True


In [ ]:
# download file
df_historical_context.to_csv('zip_rent_analysis_historical_{}.csv'.format(latest_dt_str), index=False)
files.download('zip_rent_analysis_historical_{}.csv'.format(latest_dt_str))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# End Notebook